In [8]:
!gcloud services enable aiplatform.googleapis.com

Operation "operations/acat.p2-749926295422-b9a57711-db38-4592-8bc4-0b7090ac30c2" finished successfully.


In [ ]:
%%bash
gcloud compute firewall-rules create allow-imc-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8501 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-imc-streamlit' 2> /dev/null

NAME                 NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-imc-streamlit  default  INGRESS    1000      tcp:8501        False


In [9]:
!pip install --quiet streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.0 MB/s eta 0:00:00


In [10]:
%%writefile gemini-chat.py
import streamlit as st
import vertexai, google.auth
from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    Part
)

paises_data = {
    "Polonia": {
        "capital": "Varsovia",
        "population": 38_750_000,
        "area_km2": 312_696,
        "currency": "Złoty (PLN)",
        "eu_member": True,
        "language": "Polish"
    },
    "Alemania": {
        "capital": "Berlin",
        "population": 84_120_000,
        "area_km2": 357_022,
        "currency": "Euro (EUR)",
        "eu_member": True,
        "language": "German"
    },
    "Ucrania": {
        "capital": "Kyiv",
        "population": 35_660_000,
        "area_km2": 603_628,
        "currency": "Hryvnia (UAH)",
        "eu_member": False,
        "language": "Ukrainian"
    }
}

def pais_info(pais: str):
  info = paises_data.get(pais)

  if not info:
    return {"error" : "Pais no encontrado",
            "paises disponibles":list(paises_data.keys())
    }

  return {
        "pais": pais,
        "data": info
    }

herramientas_mapeo = {"pais_info" : pais_info}

# --- Generar Modelo
credentials, project_id = google.auth.default()
vertexai.init(project=project_id, location="us-central1")

context = """
You are an expert in geography, history and politics of Central Europe.
You can answer general questions, but when population data is needed,
you must call the function numero_habitantes. When asked about other countries,
say you are not qualified to answer questions about other countries.
"""
model_name = "gemini-2.5-flash-lite"
model = GenerativeModel(model_name, system_instruction = context)


# --- Web UI con Streamlit
st.title("Chat con Gemini")
if "chat" not in st.session_state:
    st.session_state.chat = model.start_chat()
prompt = st.text_input("Ask me about Central Europe")
if st.button("Submit") and prompt.strip():
    st.chat_message("user").write(prompt)

    try:
        response = st.session_state.chat.send_message(prompt)
        part = response.candidates[0].content.parts[0]

        if part.function_call:
            fn_name = part.function_call.name
            fn_args = dict(part.function_call.args)

            if fn_name in herramientas_mapeo:
                tool_result = herramientas_mapeo[fn_name](**fn_args)

                response = st.session_state.chat.send_message(
                    Part.from_function_response(
                        name=fn_name,
                        response=tool_result,
                    )
                )

        st.chat_message("assistant").write(response.text)

    except Exception as e:
        st.error(f"Error: {e}")

Writing gemini-chat.py


In [11]:
!streamlit run gemini-chat.py --server.port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.6.30.94:8501

  Stopping...
  Stopping...
